In [173]:
from os import listdir
import cv2
import numpy as np
from skimage.util import random_noise
from imutils import grab_contours
import copy
import matplotlib.pyplot as plt

In [270]:
import numpy as np
import math

OUTPUT_NEURONS = 3
INPUT_NEURONS = 4
HIDDEN_NEURONS = 4

def Weight_Initialization():
    # Initializing of the Weights. Random float number between -0.5 to 0.5 for weights.
    np.random.seed(1)
    inputs= np.random.uniform(-0.5, 0.5, size=(5, INPUT_NEURONS))
    wji= np.random.uniform(-0.5, 0.5, size=(HIDDEN_NEURONS, INPUT_NEURONS))
    wkj = np.random.uniform(-0.5, 0.5, size=(OUTPUT_NEURONS, HIDDEN_NEURONS))
    bias_j = np.random.uniform(0, 1, size=(HIDDEN_NEURONS,1 ))
    bias_k = np.random.uniform(0, 1, size=(OUTPUT_NEURONS, 1))
    print(inputs)
    print(bias_j)
    return inputs, wji,bias_j
# def Read_Files():
#     # Reading of Segmented Training Files, and Target Files.


def Forward_Input_Hidden(inputs,wji, bias_j):
    # Forward Propagation from Input -> Hidden Layer.
    # Obtain the results at each neuron in the hidden layer.
    # Calculate 𝑁𝑒𝑡𝑗and 𝑂𝑢𝑡𝑗

    Netj = np.dot(inputs,wji.T) 
    print("Netj :")
    print(Netj)
    Outj = 1/(1 + math.e**-(Netj + np.transpose(bias_j)))
    print("Outj :")
    print(Outj)
    return Netj,Outj

def Forward_Hidden_Output(Netj,wkj, bias_k):
    # Forward Propagation from Input -> Hidden Layer.
    # Obtain the results at each neuron in the hidden layer.
    # Calculate 𝑁𝑒𝑡kand 𝑂𝑢𝑡k

    Netk = np.dot(Netj,wkj.T) 
    print("Netk :")
    print(Netk)
    Outk = 1/(1 + math.e**-(Netk + np.transpose(bias_k)))
    print("Outk :")
    print(Outk)
    return Netk, Outk


In [271]:
def Check_for_End(Outk, targets, user_set):
    # Check whether the total error is less than the error set by the user or the number of iterations is reached.
    # returns true or false
    def Error_Correction(outs, targets):
        result = ((outs - targets)**2)/2
        return result
    if Error_Correction(Outk, targets)< user_set:
        return True

    else: 
        return False
        

In [272]:
def Weight_Bias_Correction_Output(Outk, targets, OutJ):
    # Correction of Weights and Bias between Hidden and Output Layer.
    # Calculate 𝑑𝑤𝑘𝑘𝑗 and 𝑑𝑏𝑘𝑘𝑗
    dwkkj = np.multiply.outer((Outk - targets) * Outk*(1 - Outk),(OutJ))
    dbkkj = (Outk - targets) * Outk*(1 - Outk) 
    # print(dwkkj)
    return dwkkj,dbkkj

In [290]:
def Weight_Bias_Correction_Hidden(outj,outk,inputs,target,wkj):
    # Correction of Weights and Bias between Input and Hidden Layer.
    # Calculate 𝑑𝑤𝑗𝑗𝑖 and 𝑑𝑏𝑗𝑗𝑖
    skl = (outk - target) * outk*(1-outk)
    dwjji= np.multiply.outer(inputs , outj *(1 - outj) * np.dot(skl,wkj) )
    dbjii = outj *(1 - outj) * np.dot(skl,wkj)

    return dwjji, dbjii

In [293]:
def Weight_Bias_Update(wkj,dwkkj, bias_k, dbkkj, wji, dwjji,bias_j,dbjii ):
    # Saving_Weights_Bias() implemented inside
    # Update Weights and Bias.
    # Calculate 𝑤𝑘𝑘𝑗+ and 𝑏𝑘𝑘𝑗+
    n = 0.5
    wkjj = wkj - n*dwkkj
    bkkj = bias_k - n*dbkkj
    print("wk+")
    print(wkjj)
    print("bias_k+")
    print(bkkj)

    # Calculate 𝑤𝑗𝑗𝑖+ and 𝑏𝑗𝑗𝑖+
    wjji = wji - n *dwjji
    bjji = bias_j - n* dbjii
    print("wj+")
    print(wjji)
    print("bias_j+")
    print(bjji)

In [294]:
# inputs, weights, biases = Weight_Initialization()
inputs= np.array([0.2,0.5])
wji= np.array([[0.1,0.2],[0.3,0.4]])
wkj= np.array([[0.5,0.6],[0.7,0.8]])
bias_j = np.array([0.2,0.2])
bias_k = np.array([0.4,0.4])
target = np.array([0.2,0.8])

netj,outj = Forward_Input_Hidden(inputs, wji, bias_j)
netk,outk = Forward_Hidden_Output(outj, wkj, bias_k)
Error_Correction(outk, target)
dwkkj,dbkkj = Weight_Bias_Correction_Output(outk,target, outj)

dwjji, dbjii = Weight_Bias_Correction_Hidden(outj,outk,inputs,target,wkj)

Weight_Bias_Update(wkj,dwkkj, bias_k, dbkkj, wji, dwjji,bias_j,dbjii)

Netj :
[0.12 0.26]
Outj :
[0.57932425 0.61301418]
Netk :
[0.65747063 0.89593832]
Outk :
[0.74220688 0.78515061]
[1.47e-01 1.00e-04]
wk+
[[0.46994941 0.56820185]
 [0.70072558 0.80076778]]
bias_k+
[0.3481282  0.40125246]
wj+
[[0.09877858 0.19857089]
 [0.29694644 0.39642722]]
bias_j+
[0.19389289 0.19285444]
